Image registration to create a pixel map between top camera (A) and quadrant camera (B) with overlapping fields of view (FOV), where quadtant camera's FOV is a subset and a rotated version of top camera's FOV. 
1. Exrtact and Load Images: Dynamically extract random matching frames from the video frames for both Camera A and Camera B.
2. Detect Features: Use ORB (Oriented FAST and Rotated BRIEF) to detect keypoints and get descriptors. 
3. Match Features: Use a matcher (rute-Force matcher with Hamming distance) to find corresponding keypoints between the images. Keep best matches.
4. Compute Homography: Estimate the homography matrix (matrix that best transforms the matched keypoints from Camera B's frame of reference to Camera A's frame of reference) to align the images (rotation, translation, scaling).
5. Pixel map: Get the pixel map from A to B and vice versa based on this.
6. Quality checks base on homography matrix.
7. Multi-frame averaging: Perform these computation on many frames and average over individual homography matrices that pass checks for more accurate results.
8. Visualise results by applying the homography to transform Camera B's images to align with Camera A's perspective.
9. Save map, H, and visualisation.


This method seems to work better then other tried, becasue:

- can deal with low number of unique features in the image (feature matching algorithms failed)
- can deal with orientation and perspective differences (template matching (= find B within A) failed)

Homography matrix explanation:

- ( h_{11} ) and ( h_{22} ): These elements represent scaling and rotation. They affect how the image is scaled and rotated around the origin.
- ( h_{12} ) and ( h_{21} ): These elements represent shear. They affect how the image is skewed.
- ( h_{13} ) and ( h_{23} ): These elements represent translation. They affect how the image is shifted in the x and y directions.
- ( h_{31} ) and ( h_{32} ): These elements represent perspective distortion. They affect how the image is warped to simulate a change in viewpoint.
- ( h_{33} ): This element is typically 1 for normalization purposes.

In [2]:
"""Imports"""
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import random


"""Define constants"""
camera_a = 'CameraTop'
camera_b_list = ['CameraSouth', 'CameraNorth', 'CameraEast', 'CameraWest']
base_path = '/ceph/aeon/aeon'
video_path = base_path + '/data/raw/AEON3/social0.2/2024-01-31T11-28-39'
video_path_a = video_path + f'/{camera_a}/{camera_a}_2024-01-31T11-00-00.avi'
results_dir = base_path + '/code/scratchpad/Orsi/pixel_mapping/pixel_mapping_results'
os.makedirs(results_dir, exist_ok=True)

"""Define utility functions"""
def extract_frame(cap, frame_idx):
    """Extract a single frame from the video at the given index."""
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    return frame if ret else None

def save_plot(img_a, img_b, img_b_aligned, kp_a, kp_b, matches, mask, title, save_path):
    """Save the final plot with warping based on average H."""
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_a, cmap='gray')
    plt.title("Camera A (Original)")

    plt.subplot(1, 3, 2)
    plt.imshow(img_b, cmap='gray')
    plt.title("Camera B (Original)")

    plt.subplot(1, 3, 3)
    plt.imshow(img_b_aligned, cmap='gray')
    plt.title("Camera B (Aligned to A)")

    plt.savefig(save_path)
    plt.close()
    
def warp_image(img_b, H, img_a_shape):
    """
    Warp Image B to Image A's perspective using the homography matrix.
    """
    # Apply homography to warp Image B
    img_b_aligned = cv2.warpPerspective(img_b, H, (img_a_shape[1], img_a_shape[0]))
    return img_b_aligned

def save_homography_and_maps(H, map_a_to_b, map_b_to_a, camera_b):
    """Save the homography matrix and pixel maps."""
    np.save(os.path.join(results_dir, f'H_{camera_b}.npy'), H)
    np.save(os.path.join(results_dir, f'map_a_to_b_{camera_b}.npy'), map_a_to_b)
    np.save(os.path.join(results_dir, f'map_b_to_a_{camera_b}.npy'), map_b_to_a)
    print(f"Saved homography and pixel maps for {camera_b}.")

def calculate_pixel_mapping(kp_a, kp_b, H, img_shape):
    """Calculate pixel mappings between two images."""
    # Create empty maps for the pixel correspondence
    map_a_to_b = np.zeros((img_shape[0], img_shape[1], 2), dtype=np.float32)
    map_b_to_a = np.zeros((img_shape[0], img_shape[1], 2), dtype=np.float32)

    # Calculate the pixel mapping
    for y in range(img_shape[0]):
        for x in range(img_shape[1]):
            # Map from A to B
            pt_a = np.array([[x, y]], dtype=np.float32).reshape(-1, 1, 2)
            pt_b = cv2.perspectiveTransform(pt_a, H)
            map_a_to_b[y, x] = pt_b[0, 0]

            # Map from B to A
            pt_b_inv = cv2.perspectiveTransform(pt_b, np.linalg.inv(H))
            map_b_to_a[y, x] = pt_b_inv[0, 0]

    return map_a_to_b, map_b_to_a

def process_video_frames(video_path_a, video_path_b, camera_b, num_good_frames=50):
    """Process frames from two videos dynamically until 20 good frames are found."""
    cap_a = cv2.VideoCapture(video_path_a)
    cap_b = cv2.VideoCapture(video_path_b)

    total_frames = int(cap_a.get(cv2.CAP_PROP_FRAME_COUNT))
    good_homographies = []

    while len(good_homographies) < num_good_frames:
        frame_idx = random.randint(0, total_frames - 1)  # Randomly select a frame index

        # Extract frames for both cameras
        frame_a = extract_frame(cap_a, frame_idx)
        frame_b = extract_frame(cap_b, frame_idx)

        if frame_a is None or frame_b is None:
            print(f"Could not read frames at index {frame_idx}. Skipping...")
            continue

        # Convert frames to grayscale
        img_a = cv2.cvtColor(frame_a, cv2.COLOR_BGR2GRAY)
        img_b = cv2.cvtColor(frame_b, cv2.COLOR_BGR2GRAY)

        # Detect and match features
        kp_a, kp_b, matches = detect_and_match_features(img_a, img_b)
        if len(matches) < 4:  # Not enough matches to compute a homography
            print("Not enough matches. Skipping frame...")
            continue

        # Compute homography
        H, mask = compute_homography(kp_a, kp_b, matches)

        if H is not None and is_perspective_good(H) and is_rotation_good(H, camera_b) and is_shear_good(H):
            good_homographies.append(H)
            print(f"Valid Homography found for frame index {frame_idx} (total valid: {len(good_homographies)})")
        else:
            print(f"Invalid Homography at frame index {frame_idx}.")

    cap_a.release()
    cap_b.release()

    # Compute the average homography
    H_avg = np.mean(good_homographies, axis=0)
    print(f"Averaged Homography:\n{H_avg} based on {len(good_homographies)} valid homographies.")

    # Use the last valid frames for final plotting
    img_b_aligned = warp_image(img_b, H_avg, img_a.shape)

    # Calculate pixel maps
    map_a_to_b, map_b_to_a = calculate_pixel_mapping(kp_a, kp_b, H_avg, img_a.shape)

    # Save results
    save_plot(img_a, img_b, img_b_aligned, kp_a, kp_b, matches, mask, 'Final Averaged Homography',
              os.path.join(results_dir, f'final_plot_{camera_b}.png'))
    save_homography_and_maps(H_avg, map_a_to_b, map_b_to_a, camera_b)

    return H_avg, good_homographies

def detect_and_match_features(img_a, img_b, nfeatures=1000):
    """Detect and match features between two images using ORB."""
    orb = cv2.ORB_create(nfeatures=nfeatures)
    kp_a, des_a = orb.detectAndCompute(img_a, None)
    kp_b, des_b = orb.detectAndCompute(img_b, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des_a, des_b)
    matches = sorted(matches, key=lambda x: x.distance)
    return kp_a, kp_b, matches

def compute_homography(kp_a, kp_b, matches):
    """Compute the homography matrix to map Image B to Image A."""
    src_pts = np.float32([kp_b[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_a[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H, mask

def is_perspective_good(H, perspective_threshold=1e-3):
    """Check if the homography matrix is good based on perspective distortion."""
    h31, h32 = H[2, 0], H[2, 1]
    return abs(h31) <= perspective_threshold and abs(h32) <= perspective_threshold

def is_shear_good(H, shear_threshold=1):
    """Check if the homography matrix is good based on shearing distortion."""
    h12, h21 = H[0, 1], H[1, 0]
    return abs(h12) <= shear_threshold and abs(h21) <= shear_threshold

def is_rotation_good(H, camera_b, tolerance=15):
    """Check if the homography matrix has a rotation component within the expected range."""
    R = H[0:2, 0:2]
    theta = np.arctan2(R[1, 0], R[0, 0]) * 180 / np.pi
    theta = theta % 360

    if camera_b == 'CameraWest':
        expected_rotation = 0
    elif camera_b == 'CameraNorth':
        expected_rotation = 90
    elif camera_b == 'CameraEast':
        expected_rotation = 180
    elif camera_b == 'CameraSouth':
        expected_rotation = 270
    else:
        raise ValueError(f"Unrecognized camera_b: {camera_b}")

    circular_difference = min(abs(theta - expected_rotation), 360 - abs(theta - expected_rotation))
    return circular_difference <= tolerance

"""Run dynamic extraction and processing for all CameraBs"""
for camera_b in camera_b_list:
    video_path_b = video_path + f'/{camera_b}/{camera_b}_2024-01-31T11-00-00.avi'
    print(f"Processing frames for Camera B: {camera_b}")
    H_avg, homographies = process_video_frames(video_path_a, video_path_b, camera_b)


Processing frames for Camera B: CameraSouth
Valid Homography found for frame index 20336 (total valid: 1)
Valid Homography found for frame index 74986 (total valid: 2)
Valid Homography found for frame index 8182 (total valid: 3)
Valid Homography found for frame index 28994 (total valid: 4)
Valid Homography found for frame index 45982 (total valid: 5)
Valid Homography found for frame index 77143 (total valid: 6)
Valid Homography found for frame index 17101 (total valid: 7)
Valid Homography found for frame index 3644 (total valid: 8)
Valid Homography found for frame index 10989 (total valid: 9)
Valid Homography found for frame index 56574 (total valid: 10)
Valid Homography found for frame index 82450 (total valid: 11)
Valid Homography found for frame index 61677 (total valid: 12)
Valid Homography found for frame index 77668 (total valid: 13)
Valid Homography found for frame index 85064 (total valid: 14)
Valid Homography found for frame index 67017 (total valid: 15)
Valid Homography found